In [68]:
import pandas as pd
import numpy as np
import warnings
from scipy import stats
from plotly import graph_objects as go

warnings.filterwarnings('ignore')

In [69]:
df_refri_total = pd.read_csv('data/refrigerante_total_2009_2019.csv')
df_refri_sexo  = pd.read_csv('data/refrigerante_sexo_2019.csv')
df_refri_admin = pd.read_csv('data/refrigerante_administracao_2019.csv')
df_gulo_total  = pd.read_csv('data/guloseimas_total_2019.csv')
df_legum_total = pd.read_csv('data/legumes_total_2019.csv')

### Parte 1: Normalidade


In [70]:
def plot_distributions(gen_locs, gen_labels):
    fig = go.Figure()
    
    for i, (df_plot, label) in enumerate(zip(gen_locs, gen_labels)):
        
        df_plot.reset_index(drop=True, inplace=True)

        shapiro_pval = stats.shapiro(df_plot['percentual']).pvalue

        label += ' (p-val: {:.2f})'.format(shapiro_pval)

        fig.add_trace(
            go.Violin(
                x                = np.ones(df_plot.shape[0]) * i,
                y                = df_plot['percentual'],
                name             = label,
                box_visible      = True,
            )
        )

    fig.update_layout(
        width       = 1100,
        height      = 600,
        yaxis_title = 'Consumo (%)',
        yaxis       = dict(range=[0, 60]),
        xaxis       = dict(tickvals=[]),
        legend      = dict(
            orientation = 'h',
            yanchor     = 'bottom',
            x           = 0,
            y           = -0.3
        )
    )

    fig.write_image('results/distributions.pdf', engine='kaleido')
    fig.show()

In [71]:
gen_locs = (
    df_refri_total.loc[df_refri_total['ano']==2009],
    df_refri_total.loc[df_refri_total['ano']==2019],
    df_refri_sexo.loc[df_refri_sexo['sexo']=='Masculino'],
    df_refri_sexo.loc[df_refri_sexo['sexo']=='Feminino'],
    df_refri_admin.loc[df_refri_admin['administracao']=='Pública'],
    df_refri_admin.loc[df_refri_admin['administracao']=='Privada'],
    df_gulo_total,
    df_legum_total
)

gen_labels = (
    'Refrigerante: Total em 2009',
    'Refrigerante: Total em 2019',
    'Refrigerante: Público Masculino em 2019',
    'Refrigerante: Público Feminino em 2019',
    'Refrigerante: Administração Pública em 2019',
    'Refrigerante: Administração Privada em 2019',
    'Guloseimas: Total em 2019',
    'Legumes: Total em 2019'
)

plot_distributions(gen_locs, gen_labels)

### Parte 2: Testes de hipótese

In [72]:
def teste_fsnedecor_tstudent(a, b, alpha=0.05):

    print('Teste de F Snedecor:')
    
    f_var = a.var()/b.var() if a.var()>b.var() else b.var()/a.var()
    variancia_pval = 1 - stats.f.cdf(f_var, a.size-1, b.size-1)
    
    if alpha < variancia_pval:
        homocedastico = True
        
        print('\tNão há evidências para rejeitar a hipótese nula (p-val: {:.2f}).'.format(variancia_pval))
        print('\tNão é possível afirmar que as variâncias são diferentes. Logo, o teste de comparação de médias deve ser homocedástico.')
    
    else:
        homocedastico = False
        
        print('\tHá evidências para rejeitar a hipótese nula (p-val: {:.2f}).'.format(variancia_pval))
        print('\tÉ possível afirmar que as variâncias são diferentes. Logo, o teste de comparação de médias deve ser heterocedástico.')

    print('\nTeste de T Student:')
    
    media_pval = stats.ttest_ind(a, b, equal_var=homocedastico).pvalue
    
    if alpha < media_pval:
        print('\tNão há evidências para rejeitar a hipótese nula (p-val: {:.2f}).'.format(media_pval))
        print('\tNão é possível afirmar que as médias são diferentes.')
    
    else:
        print('\tHá evidências para rejeitar a hipótese nula (p-val: {:.2f}).'.format(media_pval))
        print('\tÉ possível afirmar que as médias são diferentes.')

#### Parte 2.1: Comparação entre refrigerante total 2009 e refrigerante total 2019

In [73]:
a     = df_refri_total.loc[df_refri_total['ano']==2009]['percentual'].reset_index(drop=True)
b     = df_refri_total.loc[df_refri_total['ano']==2019]['percentual'].reset_index(drop=True)
teste_fsnedecor_tstudent(a, b, alpha=0.05)

Teste de F Snedecor:
	Não há evidências para rejeitar a hipótese nula (p-val: 0.06).
	Não é possível afirmar que as variâncias são diferentes. Logo, o teste de comparação de médias deve ser homocedástico.

Teste de T Student:
	Há evidências para rejeitar a hipótese nula (p-val: 0.00).
	É possível afirmar que as médias são diferentes.


#### Parte 2.2: Comparação entre refrigerante masculino 2009 e refrigerante feminino 2019

In [74]:
a     = df_refri_sexo.loc[df_refri_sexo['sexo']=='Masculino']['percentual'].reset_index(drop=True)
b     = df_refri_sexo.loc[df_refri_sexo['sexo']=='Feminino']['percentual'].reset_index(drop=True)
teste_fsnedecor_tstudent(a, b, alpha=0.05)

Teste de F Snedecor:
	Não há evidências para rejeitar a hipótese nula (p-val: 0.45).
	Não é possível afirmar que as variâncias são diferentes. Logo, o teste de comparação de médias deve ser homocedástico.

Teste de T Student:
	Não há evidências para rejeitar a hipótese nula (p-val: 0.40).
	Não é possível afirmar que as médias são diferentes.


#### Parte 2.3: Comparação entre refrigerante pública 2019 e refrigerante privada 2019

In [75]:
a     = df_refri_admin.loc[df_refri_admin['administracao']=='Pública']['percentual'].reset_index(drop=True)
b     = df_refri_admin.loc[df_refri_admin['administracao']=='Privada']['percentual'].reset_index(drop=True)
teste_fsnedecor_tstudent(a, b, alpha=0.05)

Teste de F Snedecor:
	Não há evidências para rejeitar a hipótese nula (p-val: 0.40).
	Não é possível afirmar que as variâncias são diferentes. Logo, o teste de comparação de médias deve ser homocedástico.

Teste de T Student:
	Há evidências para rejeitar a hipótese nula (p-val: 0.02).
	É possível afirmar que as médias são diferentes.


### Parte 3: Correlação entre consumo de refrigerante, guloseimas e legumes

In [76]:
stats.pearsonr(
    df_refri_total.loc[df_refri_total['ano']==2019]['percentual'],
    df_gulo_total['percentual']
).statistic

0.43261258562841437

In [77]:
stats.pearsonr(
    df_refri_total.loc[df_refri_total['ano']==2019]['percentual'],
    df_legum_total['percentual']
).statistic

0.23549838361681327

In [78]:
stats.pearsonr(
    df_gulo_total['percentual'],
    df_legum_total['percentual']
).statistic

0.11995849134364218

In [79]:
fig = go.Figure()

fig.add_scatter(
    x    = df_refri_total.loc[df_refri_total['ano']==2019]['capital'],
    y    = df_refri_total.loc[df_refri_total['ano']==2019]['percentual'],
    name = 'Refrigerante: Total em 2019',
    mode = 'lines+markers'
)

fig.add_scatter(
    x    = df_gulo_total['capital'],
    y    = df_gulo_total['percentual'],
    name = 'Guloseimas: Total em 2019',
    mode = 'lines+markers'
)

fig.add_scatter(
    x    = df_legum_total['capital'],
    y    = df_legum_total['percentual'],
    name = 'Legumes: Total em 2019',
    mode = 'lines+markers'
)

fig.update_layout(
    width       = 1200,
    height      = 500,
    yaxis_title = 'Consumo (%)',
    yaxis       = dict(range=[0, 50])
)

fig.write_image('results/correlations.pdf', engine='kaleido')
fig.show()